In [ ]:
from parquet_data_reader import ParquetDataReader
import polars as pl
import pandas as pd
import numpy as np
from utils.process_data import process_item_data, explode_user_interactions
from parquet_data_reader import ParquetDataReader
from models.feature_based import CosineSimilarityRecommender
pl.Config.set_tbl_cols(-1)

## Data import and EDA

## Preprocessing

## Model Fit

## Model presentation